In [3]:
# !pip install geopy

In [1]:
import os
import pandas as pd 
from geopy.distance import geodesic
from tqdm import tqdm
from scipy.spatial import KDTree
import folium
from folium.plugins import MarkerCluster
from folium import PolyLine

# 설정
heatline_folder = "./make_file/gu_division/"  # 열선 데이터 폴더 경로
road_folder = "../서울시_자동차_도로_수집/make_file/gu_division/"  # 도로 데이터 폴더 경로
file_encoding = "utf-8"

# 서울시 행정구 목록
districts = [
    "강남구", "강동구", "강북구", "강서구", "관악구", "광진구", "구로구", "금천구",
    "노원구", "도봉구", "동대문구", "동작구", "마포구", "서대문구", "서초구",
    "성동구", "성북구", "송파구", "양천구", "영등포구", "용산구", "은평구",
    "종로구", "중구", "중랑구"
]

MAX_SIZE = 20

In [2]:
def get_nearby_roads(heatline_df, road_df, k=MAX_SIZE):
    # 열선의 중점 좌표를 추출 (위도, 경도)
    heatline_coords = heatline_df[["중점_위도", "중점_경도"]].values
    
    # 도로의 중간점 좌표를 계산 (시작점과 종료점의 중간)
    road_coords = road_df[["시작점_위도", "시작점_경도", "종료점_위도", "종료점_경도"]].values
    road_midpoints = (road_coords[:, :2] + road_coords[:, 2:]) / 2  # 도로의 중간점
    
    # KDTree를 사용하여 도로 중간점들의 최근접 거리를 계산
    tree = KDTree(road_midpoints)
    
    # 열선에 대해 최근접 k개의 도로를 찾기
    nearby_roads = []

    for _, heatline_row in heatline_df.iterrows():
        # 열선의 중점 좌표 (위도, 경도)
        heatline_point = [heatline_row["중점_위도"], heatline_row["중점_경도"]]
        
        # 최근접 k개의 도로 찾기 (k=5)
        distances, indices = tree.query([heatline_point], k=k)
        
        # 각 도로에 대해 결과 저장
        for i in range(k):
            road_index = indices[0][i]
            road = road_df.iloc[road_index]
            nearby_roads.append({
                'heatline_id': heatline_row['열선의_id'],
                'road_id': road['id'],
                'road_start_lat': road["시작점_위도"],
                'road_start_lon': road["시작점_경도"],
                'road_end_lat': road["종료점_위도"],
                'road_end_lon': road["종료점_경도"],
                'distance': distances[0][i]
            })
    
    return nearby_roads


In [3]:
# 구 이름으로 파일을 필터링하는 함수
def filter_files_by_district(files, district):
    return [file for file in files if district in os.path.basename(file)]

In [4]:
def process_district(district, heat_files, road_files):
    # 해당 구에 맞는 열선 파일과 도로 파일만 필터링
    heatline_files = filter_files_by_district(heat_files, district)
    road_files = filter_files_by_district(road_files, district)
    
    # 데이터프레임 리스트 생성
    heatline_df_list = [pd.read_csv(f, encoding=file_encoding) for f in heatline_files]
    road_df_list = [pd.read_csv(f, encoding=file_encoding) for f in road_files]

    if not heatline_df_list or not road_df_list:
        print(f"{district} 지역 데이터가 부족합니다.")
        return
    
    # 데이터 합치기
    heatline_df = pd.concat(heatline_df_list, ignore_index=True)
    road_df = pd.concat(road_df_list, ignore_index=True)
    
    # 최근접 도로 찾기
    filtered_roads = get_nearby_roads(heatline_df, road_df)

    # 지도 저장 경로
    save_path = f"./make_file/통합_지도/{district}" 
    
    # 지도 생성
    create_map(heatline_df, filtered_roads, save_path)


In [5]:
# 구에 대해 반복 작업을 수행하는 함수
def main():
    for district in districts:
        process_district(district, heat_files, road_files)


In [6]:
def create_map(heatline_df, filtered_roads, temp_save_path):
    # 기본 지도 생성
    m = folium.Map(location=[37.5665, 126.9780], zoom_start=12) 
    
    # 열선 마커 추가 및 빨간 선으로 시작점과 끝점 연결
    for _, row in heatline_df.iterrows():
        start = [row["시점_위도"], row["시점_경도"]]
        end = [row["종점_위도"], row["종점_경도"]]
        
        # 열선 시작과 끝을 빨간 선으로 연결
        folium.PolyLine([start, end], color='red', weight=3, opacity=1).add_to(m)
        
        # 열선의 중간점 계산
        midpoint = [(start[0] + end[0]) / 2, (start[1] + end[1]) / 2]
        
        # 열선 중간점에 마커 추가 (중앙에만 마커 추가)
        folium.Marker(midpoint, popup=f"열선 {row['열선의_id']}", icon=folium.Icon(color='red')).add_to(m)
    
    # 도로 선 그리기 (필터링된 도로만 그리기)
    for road in filtered_roads:
        road_start = [road['road_start_lat'], road['road_start_lon']]
        road_end = [road['road_end_lat'], road['road_end_lon']]
        
        # 도로 중간점 계산
        road_midpoint = [(road_start[0] + road_end[0]) / 2, (road_start[1] + road_end[1]) / 2]
        
        # 도로 선 그리기
        folium.PolyLine([road_start, road_end], color='blue', weight=2.5, opacity=1).add_to(m)
        
        # 도로 중간점에 마커 추가
        folium.Marker(road_midpoint, popup=f"{road['road_id']}", icon=folium.Icon(color='blue')).add_to(m)

    len_size = len(heatline_df)
    # 지도 저장 경로
    save_path = temp_save_path + f"_{len_size}_통합지도.html"
    
    # 지도 저장
    m.save(save_path)
    print(save_path)

In [7]:
# 조건에 맞는 파일 목록을 미리 로드
heat_files = []
for file_name in os.listdir(heatline_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(heatline_folder, file_name)  # 안전한 경로 조합
        try:
            heat_files.append(file_path)
        except Exception as e:
            print(f"파일 {file_name} 처리 중 오류 발생: {e}")

road_files = []
for file_name in os.listdir(road_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(road_folder, file_name)  # 안전한 경로 조합
        try:
            road_files.append(file_path)
        except Exception as e:
            print(f"파일 {file_name} 처리 중 오류 발생: {e}")


In [8]:
# 구별로 작업 실행
main()

./make_file/통합_지도/강남구_59_통합지도.html
./make_file/통합_지도/강동구_5_통합지도.html
./make_file/통합_지도/강북구_14_통합지도.html
./make_file/통합_지도/강서구_13_통합지도.html
./make_file/통합_지도/관악구_26_통합지도.html
./make_file/통합_지도/광진구_33_통합지도.html
./make_file/통합_지도/구로구_11_통합지도.html
./make_file/통합_지도/금천구_15_통합지도.html
./make_file/통합_지도/노원구_8_통합지도.html
./make_file/통합_지도/도봉구_36_통합지도.html
./make_file/통합_지도/동대문구_13_통합지도.html
./make_file/통합_지도/동작구_45_통합지도.html
./make_file/통합_지도/마포구_21_통합지도.html
./make_file/통합_지도/서대문구_26_통합지도.html
./make_file/통합_지도/서초구_19_통합지도.html
./make_file/통합_지도/성동구_48_통합지도.html
./make_file/통합_지도/성북구_53_통합지도.html
./make_file/통합_지도/송파구_2_통합지도.html
./make_file/통합_지도/양천구_17_통합지도.html
./make_file/통합_지도/영등포구_8_통합지도.html
./make_file/통합_지도/용산구_20_통합지도.html
./make_file/통합_지도/은평구_6_통합지도.html
./make_file/통합_지도/종로구_14_통합지도.html
./make_file/통합_지도/중구_31_통합지도.html
./make_file/통합_지도/중랑구_26_통합지도.html
